## Data Cleaning (Appendix, Draft)

Project FeederWatch is a citizen-science-based data source supported by the Cornell Lab of Ornithology, which collects observations of bird species at backyard feeders and habitats all over the world in an annual November-April survey.

Our raw file comes from the [Project FeederWatch](https://feederwatch.org/explore/raw-dataset-requests/) 2021 New York checklist data and site description data. This file is extremely large and has sightings from about November 2020 to April 2021. 

In [2]:
# importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
import scipy.stats as stats

In [3]:
# importing sql
%load_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:

## Joining Original Dataset (not zero-filled) with Sites

Our first dataset joins the original dataset of 2021 Season Feederwatch Observations with the sites dataset with location information. 

In [5]:
# reading in original dataset from Feederwatch for observations
total_df = pd.read_csv("PFW_2021_public.csv")

We chose to drop several columns in the observations dataset (`total_df`) such that only `loc_id`, `subnational1_code`, and `species_code` are present. We also chose to eliminate most columns from the sites dataset (`sites_df`) so that only `loc_id` and `housing_density` remain, allowing for an efficient data join. 

In [6]:
total_df.columns
total_df = total_df.drop(['latitude', 'longitude',
       'entry_technique', 'sub_id', 'obs_id', 'Month', 'Day', 'Year',
       'PROJ_PERIOD_ID', 'how_many', 'valid', 'reviewed',
       'day1_am', 'day1_pm', 'day2_am', 'day2_pm', 'effort_hrs_atleast',
       'snow_dep_atleast', 'Data_Entry_Method'], axis = 1)
total_df

,loc_id,subnational1_code,species_code
0,L12782033,CA-ON,amtspa
1,L12782033,CA-ON,blujay
2,L12782033,CA-ON,bkcchi
3,L12755941,CA-SK,dowwoo
4,L12755941,CA-SK,whbnut
...,...,...,...
2897100,L2404002,US-NY,daejun
2897101,L2404002,US-NY,amerob
2897102,L2404002,US-NY,houfin
2897103,L2404002,US-NY,norcar


In [9]:
# reading in original dataset from Feederwatch for locations
sites_df = pd.read_csv("PFW_count_site_data_public_2021.csv")

sites_df = sites_df.drop(['proj_period_id', 'yard_type_pavement', 'yard_type_garden',
       'yard_type_landsca', 'yard_type_woods', 'yard_type_desert',
       'hab_dcid_woods', 'hab_evgr_woods', 'hab_mixed_woods', 'hab_orchard',
       'hab_park', 'hab_water_fresh', 'hab_water_salt', 'hab_residential',
       'hab_industrial', 'hab_agricultural', 'hab_desert_scrub',
       'hab_young_woods', 'hab_swamp', 'hab_marsh', 'evgr_trees_atleast',
       'evgr_shrbs_atleast', 'dcid_trees_atleast', 'dcid_shrbs_atleast',
       'fru_trees_atleast', 'cacti_atleast', 'brsh_piles_atleast',
       'water_srcs_atleast', 'bird_baths_atleast', 'nearby_feeders',
       'squirrels', 'cats', 'dogs', 'humans',
       'fed_yr_round', 'fed_in_jan', 'fed_in_feb', 'fed_in_mar', 'fed_in_apr',
       'fed_in_may', 'fed_in_jun', 'fed_in_jul', 'fed_in_aug', 'fed_in_sep',
       'fed_in_oct', 'fed_in_nov', 'fed_in_dec', 'numfeeders_suet',
       'numfeeders_ground', 'numfeeders_hanging', 'numfeeders_platfrm',
       'numfeeders_humming', 'numfeeders_water', 'numfeeders_thistle',
       'numfeeders_fruit', 'numfeeders_hopper', 'numfeeders_tube',
       'numfeeders_other', 'population_atleast',
       'count_area_size_sq_m_atleast'], axis = 1)
sites_df.head()

,loc_id,housing_density
0,L100016,2.0
1,L100016,2.0
2,L100016,2.0
3,L100016,2.0
4,L100016,2.0


The joined dataframe `join_df` combines `total_df` and `sites_df` using an `INNER JOIN` on `loc_id`, which provides us with information about the environment in which the observation entry took place. By doing this, we lose some data entries because their location is not described in `sites_df`. 

In [8]:
join_df = total_df.set_index('loc_id').join(sites_df.set_index('loc_id'), on = "loc_id", how="inner")

In [15]:
#creates a list of all subnational1_code in the dataset 
regions_list = total_df['subnational1_code'].unique()

plot_df = pd.DataFrame(columns = ['housing_density', 'count'])

def plot_species_counts(state_name):
    df = join_df[join_df['subnational1_code'] == state_name]
    
    x = ['Rural', 'Subrural', 'Suburban', 'Urban']
    counts_list = []
    
    rural_df = df[df["housing_density"] == 1.0]
    subrural_df = df[df["housing_density"] == 2.0]
    suburban_df = df[df["housing_density"] == 3.0]
    urban_df = df[df["housing_density"] == 4.0]
    
    if len(rural_df) > 0 and len(subrural_df) > 0 and len(suburban_df) > 0 and len(urban_df) > 0:
        rural_count = len(rural_df['species_code'].unique())
        subrural_count = len(subrural_df['species_code'].unique())
        suburban_count = len(suburban_df['species_code'].unique())
        urban_count = len(urban_df['species_code'].unique())

        counts_sum = rural_count + subrural_count + suburban_count + urban_count

        counts_list.append(rural_count / counts_sum)
        counts_list.append(subrural_count / counts_sum)
        counts_list.append(suburban_count / counts_sum)
        counts_list.append(urban_count / counts_sum)

        plot_df.loc[len(plot_df)] = [1.0, rural_count / counts_sum]
        plot_df.loc[len(plot_df)] = [2.0, subrural_count / counts_sum]
        plot_df.loc[len(plot_df)] = [3.0, suburban_count / counts_sum]
        plot_df.loc[len(plot_df)] = [4.0, urban_count / counts_sum]
    
#         sns.scatterplot(x = x, y = counts_list)

for state in regions_list:
    plot_species_counts(state)
    
plot_df.head()

,housing_density,count
0,1.0,0.288845
1,2.0,0.252988
2,3.0,0.252988
3,4.0,0.205179
4,1.0,0.311377


In [ ]:
new_df = pd.concat([plot_df, pd.get_dummies(plot_df['housing_density'], drop_first = True,)], axis=1)

In [ ]:
# RUN WHEN READY TO EXPORT
# new_df.to_csv('all_housing_density.csv')

## Creating full zero-filled csv for correlation exploration in final paper

We first took the original dataset `PFW_2021_public.csv` and sliced it to only include data from NY in order to cut down our dataset. Then, we used an R function provided by FeederWatch to conduct taxonomic roll-up and zero-filling, two procedures recommended by FeederWatch to limit errors. The R code used to clean is provided [here](https://engagement-center.github.io/Project-FeederWatch-Zerofilling-Taxonomic-Rollup-Public/).

1. *Taxonomic roll-up*: A process of combining observations that were recorded under different species codes but would best be treated as the same species. For example, some observers may take note of subspecies, which are then recorded under different codes than the overall species when they should logically be combined.

2. *Zero-filling*: adding counts of 0 for all species that were not recorded at an observation, essential for accounting for the fact that observation data is inherently presence-only.

The resulting csv is `rolled_up_NY_df.csv`, which is then additionally processed as follows:

In [ ]:
# reading in raw provided data, zero-filled already, only data from NY
csv = pd.read_csv("rolled_up_NY_df.csv")

In [ ]:
# making dataframe
df = pd.DataFrame(csv)

In [ ]:
new_columns = list(map(str.lower, df.columns))
df.columns = new_columns

We decided to drop `latitude` and `longitude`. We also dropped irrelevant columns, such as `ENTRY_TECHNIQUE` (a variable indicating method of site localization), `PROJ_PERIOD_ID` (calendar year of end of FeederWatch season), `sub_id` and `obs_id` (indentifiers for checklist or species respectively), `effort_hrs_atleast` (survey time), and `DATA_ENTRY_METHOD` (web/mobile/paper). Only valid observations were kept.

In [ ]:
# dropping irrelevant columns
df.drop(['unnamed: 0', '...1', 'latitude', 'longitude', 'entry_technique', 'proj_period_id', 'reviewed', 'sub_id', 'obs_id',
        'effort_hrs_atleast', 'data_entry_method'], axis= 1, inplace= True)

In [ ]:
# dropping observations that are not valid
df = df[df['valid'] == 1]

In [ ]:
df.head()

By convention, bird species are stored as 6-letter codes. However, this makes readability and interpretability more difficult later on. To remedy this, we can do an inner join with a taxonomy table provided by FeederWatch so we can add a column with the species full common name.

In [ ]:
# joining common names
df = pd.DataFrame(pd.read_csv("PFW-species-translation-table.csv"))
%sql df << SELECT loc_id, subnational1_code, month, day, year, df.species_code, how_many, valid, day1_am, day1_pm, day2_am, day2_pm, snow_dep_atleast, scientific_name, american_english_name AS species_name FROM df INNER JOIN species_translate_df ON df.species_code = species_translate_df.species_code;

In [ ]:
# dropping rows where snow depth was null
df = df.dropna(subset=['snow_dep_atleast'])
# creating new category with string corresponding to each value in snow depth (for binning in the line plots)
df['snow_category'] = 'No_Snow'
df.loc[df['snow_dep_atleast'] == 0.001, 'snow_category'] = 'Light_Snow'
df.loc[df['snow_dep_atleast'] == 5.000, 'snow_category'] = '5 to 15 cm'
df.loc[df['snow_dep_atleast'] == 15.001, 'snow_category'] = 'Heavy_Snow'
snow_dummies = pd.get_dummies(df['snow_category'], drop_first=True)
df = pd.concat([df, snow_dummies], axis=1)

# dropping rows where housing density is null
df = join_df.dropna(subset=['housing_density'])
# creating new category with string 
df['housing_density_bins'] = 'rural'
df.loc[df['housing_density'] == 2.0, 'housing_density_bins'] = "rural/suburban"
df.loc[df['housing_density'] == 3.0, 'housing_density_bins'] = "suburban"
df.loc[df['housing_density'] == 4.0, 'housing_density_bins'] = "urban"
density_dummies = pd.get_dummies(df['housing_density_bins'], drop_first=True)
df = pd.concat([df, density_dummies], axis=1)

We also created a joined dataframe `zero_filled_join_df` that combines `df` and `sites_df` using an `INNER JOIN` on `loc_id`, which provides us with information about the environment in which the observation entry took place. By doing this, we lose about half of our `df` data entries because their location is not described in `sites_df`. 

In [ ]:
%sql zero_filled_join_df << SELECT month, day, year, df.loc_id, species_code, species_name, how_many, day1_am, day1_pm, day2_am, day2_pm, Light_Snow, Heavy_Snow, No_Snow, proj_period_id, yard_type_pavement, yard_type_garden, yard_type_landsca, yard_type_woods, yard_type_desert,hab_dcid_woods, hab_evgr_woods, hab_mixed_woods, hab_orchard, hab_park, hab_water_fresh, hab_water_salt, hab_residential,hab_industrial, hab_agricultural, hab_desert_scrub, hab_young_woods, hab_swamp, hab_marsh, brsh_piles_atleast, water_srcs_atleast, bird_baths_atleast, nearby_feeders, squirrels, cats, dogs, humans, housing_density, rural/suburban, suburban, urban, population_atleast, scientific_name, species_name FROM df INNER JOIN sites_df ON df.loc_id = sites_df.loc_id;

In [ ]:
zero_filled_join_df.head()

In [ ]:
## RUN WHEN READY TO EXPORT
# zero_filled_join_df.to_csv('zero_filled_join.csv')

## Creating Accipiter and Junco csvs

The Accipiter and Junco CSVs are sliced versions of `zero_filled_join_df` that only include entries with genus "Accipiter" or "Junco" respectively.

In [ ]:
# splitting the scientific name into two columns - the genus and species names
zero_filled_join_df[['genus','species']] = zero_filled_join_df['scientific_name'].str.split(expand=True).iloc[:, :2]

In [ ]:
zero_filled_join_df['genus'].value_counts().head()

We chose Accipiter and Junco since they are distinct and differing species with a high number of observations in our data. In the final paper, there is a more in-depth explaination for this choice. 

In [ ]:
# selecting Accipiter genus data
accipiter_df = zero_filled_join_df[zero_filled_join_df['genus'] == 'Accipiter']

# selecting Junco genus data
junco_df = zero_filled_join_df[zero_filled_join_df['genus'] == 'Junco']

In [ ]:
# select only certain columns that we need for futher analysis
accipiter_df = accipiter_df[['loc_id', 'genus', 'species', 'how_many', 'species_name', 'housing_density', 'rural/suburban', 'suburban', 'urban', 'population_atleast']]
junco_df = junco_df[['loc_id', 'genus', 'species', 'how_many', 'species_name', 'housing_density', 'rural/suburban', 'suburban', 'urban', 'population_atleast']]

In [ ]:
## RUN WHEN READY TO EXPORT
# accipiter_df.to_csv('accipiter_df.csv')
# junco_df.to_csv('junco_df.csv')

# STUFF TO MOVE TO OTHER APPENDIX

In [ ]:
## ONLY RELEVANT FOR REDPOLLS, BUT NOTE THAT THE ABOVE CSVS DO 
## USE DUMMY VARIABLES FOR SNOW EVEN THOUGH WE DON'T REALLY USE THEM LATER 

# dropping rows where snow depth was null
snow_df = species_limited_df.dropna(subset=['snow_dep_atleast'])

# creating new category with string corresponding to each value in snow depth (for binning in the line plots)
snow_df['snow_category'] = 'No Snow'
snow_df.loc[snow_df['snow_dep_atleast'] == 0.001, 'snow_category'] = '< 5 cm'
snow_df.loc[snow_df['snow_dep_atleast'] == 5.000, 'snow_category'] = '5 to 15 cm'
snow_df.loc[snow_df['snow_dep_atleast'] == 15.001, 'snow_category'] = '> 15 cm'

In [ ]:
## THIS IS USED TO MAKE THE BLUEJAY_DF.CSV, ONLY USED IN THE OTHER APPENDIX FOR MARY'S REGRESSIONS

# making a list of the 10 most frequently observed species
frequent_species = df['species_code'].value_counts()[:1].index

# creating new dataframe limited to just the most frequent species observations
species_limited_df = df[df['species_code'].isin(frequent_species)]

species_limited_df.to_csv("bluejay_df.csv")